
De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3n} \left [ f(x_0) + f(x_n) + 2 \sum_{i=1}^{\frac{n}{2}-1}f(x_{2i}) + 4 \sum_{i=1}^{\frac{n}{2}}f(x_{2i-1})\right ] $$
con $h=b-a$ y $n$ número de subintervalos (par).

Nota: Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos. Estos índices se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices que se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [5]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [7]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [1]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

**¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?**

En el caso de $p$, que es el número de subprocesos a lanzar, éste se recomienda que sea menor o igual al número de cores que el sistema operativo puede usar.
Para conocer el número de cores se utiliza la función cpu_count de multiprocessing.

In [2]:
mp.cpu_count()

4

A la variable $n$, el número de subintervalos, se le debe colocar como restricción que sea par, pues es una restricción en la fórmula de la regla de Simpson.

En cuanto a $ns\_p$, el número de subintervalos que procesa cada subproceso, se recomienda repartirlos equitativamente entre cada subproceso. En este ejercicio se define como $\frac{nt}{p}$ cumpliendo así la recomendación.

Adicionalmente, se debe de tener cuidado que $ns\_p$ sea un número entero (antes de aplicarle la función $int$), esto se logra mediante la combinación de los valores que toman $p$ y $n$. Por ejemplo si definimos $p=3$ y $n=8$, entonces $np\_s = 5.33$ y por lo tanto $int(ns\_p) = 5$, esto ocasiona que se evalúen 15 subintervalos en lugar de los 16 que indica $nt$ obteniendo así resutados incorrectos. En cambio si definimos $p$ = 4 y $n=8$, $np\_s = 4$ y de esta forma sí se evalúan los 16 intervalos.

---------------------------------------------------------------------------------------------------

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

In [4]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [3]:
p=2 #número de subprocesos a lanzar

In [4]:
n=8 #número de subintervalos a lanzar
h_hat=h/n

In [5]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [6]:
nt

16

In [7]:
ns_p

8.0

In [8]:
ns_p=int(ns_p)

In [9]:
ns_p

8

In [10]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

**En el bloque del with ¿por qué tenemos que dividir por $\frac{}{6n}$ en lugar de por $\frac{}{3n}$ y por qué tenemos que restar $f(b)$?**

Revisando a detalle la función *Scf_parallel*, se observa que se están repartiendo los nodos entre los subprocesos como nodos continuos de forma equitativa. Por ello, en la evaluación de nodos se incluye el último nodo de cada subintervalo (range termina en end+1) pues de lo contrario habría huecos en las evaluciones. Por ejemplo si $p=2$ y $n=6$, se reparten los nodos de 1 a 6 en un subproceso y de 7 a 12 en el otro subproceso, si no se incluyera end+1, no se estaría evaluando el nodo 6. 

De esta forma, el último nodo del intervalo completo, se está incluyendo en el for y por lo tanto ya se está evaluando el nodo $f(b)$, y no solo eso, además se está multiplicando por 2 el nodo $f(b)$ tal y como se hace con todos los nodos pares. Es decir, ya se incluye $2*f(b)$ en $sum2$. Por esto es necesario restar una vez $f(b)$ al final en $aprox$, para incluir solamente una vez el término $f(b)$ como lo indica la fórmula.

En el caso de $f(a)$, esto no ocurre pues el nodo cero no se incluye en ninguna evaluación de los for.

En cuanto a por qué se divide entre $6n$ en lugar de entre $3n$, es porque en realidad se están evaluando $2*n=nt$ nodos en el algoritmo, por ello debe dividirse entre $3$ multiplicado por el número de nodos que se están evaluando. Es decir entre $3*(2*n) = 6n$.

---------------------------------------------------------------------------------------------------

In [11]:
aprox

0.7468242574357303

In [12]:
obj, err = quad(f, a, b)

In [13]:
obj

0.7468241328124271

In [14]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [15]:
err_relativo(aprox,obj)

1.668710178971902e-07

**¿Qué pasa si $n$ no es par? ¿qué tendríamos que modificar en la implementación si no es par?**

Si $n$ no es par, los evaluación de cada subconjunto de índices en cada subproceso van cambiando de factor. Por ejemplo, en el caso anterior para $n=6$, el primer subconjunto del primer proceso incluye el $1,3,5$ y el primer subconjunto del segundo proceso al $7,9,11$, siendo congruente y pudiendo multiplicar ambos por $4$. Sin embargo al ser $n=5$, el primer subconjunto del primer proceso incluiría el $1,3,5$, pero el primer subconjunto del segundo proceso incluiría el $6,8,10$, teniendo que modificar por distintos factores en cada subproceso.

Es decir, si $n$ no es par, podría suceder que el valor de *begin* en los subprocesos diferentes al $0$ sea par, y por lo tanto los nodos pares se tomarían como si fueran pares y viceversa, pues por la forma en que está implementado el algoritmo, se asume que el nodo *begin* es impar y el nodo *begin+1* es par siempre. De esta forma se estarían multiplicando de forma incorrecta por 4 los nodos pares y por 2 los impares, obteniendo así un resultado incorrecto en *aprox*.

Una forma de corregirlo es modificando en la implementación la forma en la cual se define *begin* y *end* asegurando que begin siempre sea impar y asegurando que todos los nodos se evalúen, por ejemplo como se muestra en *Scf_parallel_impar* a continuación.

In [5]:
p=2 #número de subprocesos a lanzar

In [6]:
n=9 #número de subintervalos a lanzar
h_hat=h/n

In [7]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [8]:
nt

18

In [9]:
ns_p

9.0

In [10]:
ns_p=int(ns_p)

In [11]:
ns_p

9

In [13]:
def Scf_parallel_impar(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    
    if mi_id==0:
        begin = (mi_id*ns_p)+1
        end = begin-1 +ns_p-1
    else:
        begin = (mi_id*ns_p)
        end = begin+ns_p

    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
        print(i)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel_impar,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

1
9
3
11
5
13
15
17
7


In [14]:
aprox

0.7468242106299985

In [15]:
obj, err = quad(f, a, b)

In [16]:
obj

0.7468241328124271

In [17]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [18]:
err_relativo(aprox,obj)

1.041979871400868e-07